In [35]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [36]:
### Step 1. 크롤링할 블로그 url 수집

driver=webdriver.Chrome('/Users/leesh970930/Desktop/chromedriver')


driver.get("http://www.naver.com")
time.sleep(2)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys('오늘일기')
element.submit()
time.sleep(3)

In [37]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click()
time.sleep(0.5)

# 검색옵션 확인
item_li = driver.find_elements_by_css_selector(".option .txt")

for i in range(0, len(item_li)):
    print(item_li[i].text)

# 검색기간 '3개월' 클릭
print(item_li[9].text)

item_li[9].click()

전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기
1개월


In [38]:
# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver)
    i = i+1

# 블로그 글 url들 수집
url_list = []
title_list = []

In [39]:
# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

print("url개수: ", len(url_list))
print("title개수: ", len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

url개수:  330
title개수:  330


In [40]:
### Step 2. 블로그 내용 크롤링

# URL이 저장된 리스트 불러오기
for url_load in url_list:
        print(url_load)
        
print("수집한 url 개수: ")
print(len(url_list))

https://blog.naver.com/theone0906?Redirect=Log&logNo=222592579453
https://blog.naver.com/sungeunbboz?Redirect=Log&logNo=222593719837
https://blog.naver.com/niveusrosa?Redirect=Log&logNo=222593518952
https://blog.naver.com/arigaddo_s2?Redirect=Log&logNo=222591481945
https://blog.naver.com/ochawonjiha?Redirect=Log&logNo=222593817323
https://blog.naver.com/freeidtime?Redirect=Log&logNo=222592666084
https://blog.naver.com/virgin99?Redirect=Log&logNo=222587331916
https://blog.naver.com/dailycooking?Redirect=Log&logNo=222592903612
https://blog.naver.com/yeomya?Redirect=Log&logNo=222592990784
https://blog.naver.com/mjymodel?Redirect=Log&logNo=222593324304
https://blog.naver.com/honey1975?Redirect=Log&logNo=222580489103
https://blog.naver.com/ariel_0402?Redirect=Log&logNo=222578161246
https://blog.naver.com/shinej123?Redirect=Log&logNo=222590489400
https://blog.naver.com/sungeunbboz?Redirect=Log&logNo=222589387489
https://blog.naver.com/theone0906?Redirect=Log&logNo=222584529003
https://blog.n

In [ ]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇 선언
number = len(url_list)    # 수집할 글 갯수
 

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_list[i]
    driver = webdriver.Chrome('/Users/leesh970930/Desktop/chromedriver')
    driver.get(url)
    
  
    
    # 크롤링
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                        
        tit = driver.find_element_by_css_selector(overlays)
        title = tit.text

        # 글쓴이 크롤링
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        date = driver.find_element_by_css_selector(overlays)
        datetime = date.text
        
         # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
        
        # 해쉬태그 크롤링
        #overlays = "_param(false|false|4)" 
        #hashtag = driver.find_element_by_css_selector(overlays)
        #hashtag = hashtag.text

    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['contents'] = content_str
        
        #target_info['hashtag'] = hashtag

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링 성공하면 글 제목을 출력
        print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
        


<ipython-input-41-1811a7cf5445>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


0 [오늘 일기] 오늘 하루
1 오늘일기 211210 엄빠 오셨다 가신 날
2 2021-12-12 오늘 일기 #운중동 
3 2021.11.29 ~ 2021.12.05 @ 라이프로그, 단양여행, 일상, 노티드 오픈런, 주간일기, 단양2박3일, 오늘일기
4 12월12일 오늘일기_일요일/새치커버 흑갈색 염색하러 사직동/전포 NC킴스클럽 구경/이전한 서면 그집뱃살/오늘 와인한잔 하루끝
5 오늘일기 _ 과거는 과거일뿐


In [34]:
print('수집한 글 갯수: ', len(dict))


# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df.head(314)


수집한 글 갯수:  16


,title,nickname,datetime,contents
0,부산영상편집학원 강의로 공부하다!,워ㅍWorkingProfiler,2021. 12. 9. 13:00,"안녕하세요. 전 애니메이션을 전공을 하였고, 현재는 휴학 중에 있습니다. 그래서 이..."
1,유튜브 음원추출과 영상 다운로드 및 동영상 저장 쉽게 하는 방법은,모이모이,2021. 12. 1. 23:53,사무실에 있는 대부분의 시간 동안 조용한 연주곡을 들으면서 일을 하고 있습니다. 에...
2,무료 영상 편집 프로그램 필모라X 사용기 초보자도 손쉽게 뚝딱 완성!,귤랭이,2021. 12. 1. 13:30,안녕하세요 귤랭이입니다. 개인의 취향과 재능이 존중받는 사회로 변화하면서 자신이 좋...
3,유튜브 음원추출 사이트 영상 다운로드 변환기,함영민,2021. 12. 8. 15:00,"공\n중파 TV에서 종편의 트로트 프로그램으로, 그리고 Youtube를 접한 이후로..."
4,인텔 i7 + 지포스 rtx2060 영상 편집 조립 컴퓨터,연우컴퓨터,2021. 12. 6. 13:48,"안녕하세요\n요즘은 많은 분들이\n미디어 편집(영상, 사진 등)에 많은 관심을 가지..."
5,영상편집학원 처음부터 탄탄하게,알통맨닷컴\n(altongman10),2021. 11. 29. 18:08,예전에는 하루하루 회사를 다니면서 \n맞겨진 일을 해결하는데 촛점이 맞춰져있었어요....
6,의료 영상의 안전하고 정확한 획득을 위한 팬텀,과학기술정보통신부,2021. 12. 9. 15:48,"팬텀(phantom) 하면 어떤 것이 떠오르시나요? 가수나 게임 캐릭터, 오페라의 ..."
7,음악앱 추천하는 유튜브 뮤직 광고 제거 영상까지 한꺼번에 즐기자,소라윙즈,2021. 12. 6. 8:30,얼마전까지만 해도 YouTube를 그리 즐겨보는 편이 아니었던지라 중간에 광고가 좀...
8,가성비 최고!! 프리미어 프로 영상편집 컴퓨터 사양,벡스컴,2021. 11. 18. 12:27,안녕하세요~ 벡스컴입니다. \n오늘 소개해드릴 PC는 요즘 많은 분들께서 어떻게 맞...
9,"슬금슬금 오르는 구독료, 맘 편히 영상 볼까나",테크플러스,2021. 11. 25. 14:49,넷플릭스가 구독료를 인상했습니다. 한국 시장에 진출한 지 5년 만에 단행한 조치에요...


In [26]:
result_df.to_excel("영상.xlsx")
result_df.to_csv("영상.csv")